# Monte Carlo Simulation for Pricing and Risk Analysis of Exotic Options

*Author*: Hamed Sedky

*Date*: 18 Janurary 2026

## Introduction

Path-dependent options, whose payoffs depend on the entire price trajectory rather than just the terminal price, pose significant challenges for analytical pricing methods. Options such as Asian options (averaging), barrier options (knock-in/knock-out), and lookback options (extrema) generally lack closed-form solutions, making Monte Carlo simulation a natural approach.
This project implements a Monte Carlo framework to price exotic path-dependent options under the Black-Scholes framework. We make two key assumptions:

1. Asset prices follow Geometric Brownian Motion (GBM)
2. Options are priced as discounted expected payoffs under the risk-neutral measure

We implement variance reduction techniques to improve computational efficiency. The implementation is validated against analytical benchmarks where available.
Objectives:
* Implement a flexible Monte Carlo engine for path-dependent derivatives
* Demonstrate variance reduction techniques
* Analyze pricing accuracy and computational performance
* Calculate option sensitivities (Greeks)

## Theory

### Black-Scholes Model

In the Black–Scholes model, we consider a financial market consisting of a risky asset with price process $S$ and a risk-free asset, typically a money market account or government bond, with value process $B$. These are assumed to follow the stochastic differential equations
$$
\begin{align}
\mathrm{d}B_t &= r B_t , \mathrm{d}t, \\
\mathrm{d}S_t &= \mu S_t , \mathrm{d}t + \sigma S_t , \mathrm{d}W_t,
\end{align}
$$
where $r$ denotes the constant risk-free interest rate, $\mu$ the expected rate of return of the asset, and $\sigma$ its volatility.

Let a derivative contract be defined by a payoff functional $\Pi$, and suppose the option expires at time $T>0$. The terminal payoff to the holder is then
$$
\Pi\left((S_t)_{t \in [0, T]}\right).
$$
Denoting by $V$ the time-zero price of the option, the absence of arbitrage implies that its fair value under the Black–Scholes assumptions is given by
$$
V = e^{-rT}\, \mathbb{E}^{\mathbb{Q}}\left[\Pi\left((S_t)_{t \in [0, T]}\right)\right],
$$
where $\mathbb{Q}$ is the risk-neutral probability measure.

For terminal payoff options, those whose payoffs depend only on $S_T$, closed-form pricing formulas may be derived in certain cases, most notably for European calls and puts. In contrast, path-dependent payoffs generally preclude analytical solutions, necessitating numerical methods such as binomial trees, finite-difference schemes, or Monte Carlo simulation.

More generally, for an asset price following an arbitrary stochastic differential equation of the form
$$
\mathrm{d}S_t = \mu(t, S_t) , \mathrm{d}t + \sigma(t, S_t) , \mathrm{d}W_t,
$$
the dynamics under the risk-neutral measure are given by
$$
\mathrm{d}S_t = r S_t , \mathrm{d}t + \sigma(t, S_t) , \mathrm{d}W_t^{\mathbb{Q}}.
$$

### Monte Carlo simulation

In a Monte Carlo setting, sample paths of the underlying asset price are generated directly from the assumed stochastic process. Discretizing the time interval $[0, T]$ into $n$ steps and simulating $N$ independent paths yields a matrix of simulated values
$$
\begin{bmatrix}
S_0^{(1)} & S_1^{(1)} & \cdots & S_n^{(1)} \\
\vdots & \vdots & \ddots & \vdots \\
S_0^{(N)} & S_1^{(N)} & \cdots & S_n^{(N)}
\end{bmatrix},
$$
where $S_k^{(m)}$ denotes the asset price at the $k^{\text{th}}$ time step along the $m^{\text{th}}$ simulated path.

Given these simulated paths, the Monte Carlo estimator for the option price is
$$
\hat{V} = \frac{e^{-rT}}{N} \sum_{m=1}^N \Pi\left(\left(S_i^{(m)}\right)_{i=0, \dots, n}\right).
$$
Under standard regularity conditions, this estimator is unbiased and converges to the true option value as $N \to \infty$, with statistical error decreasing at rate $\mathcal{O}(N^{-1/2})$.

## Implementation

The core implementation resides in the `pricing` package, which is expected to be located at the same directory level as the `notebooks` folder containing this notebook. If import errors occur, it may be necessary to adjust the Python path accordingly before importing the `pricing` module.

In [7]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parent
sys.path.append(str(PROJECT_ROOT))

In [13]:
import pricing.markets as markets
from pricing.solvers import exact_gbm
import pricing.options as options
import pricing.engines as engines
import pricing.black_scholes as bs

As a baseline validation, we price a European call option using Monte Carlo simulation and compare the result against the analytical Black–Scholes formula.

In [48]:
# Model variables
r = 0.05
sigma = .1
s0 = 100
T = 5
K = 100

# Simulation variables
n_steps = 1
n_paths = 10 ** 6
seed = 100
antithetic = True

market = markets.BlackScholesMarket(r = r, sigma = sigma, solver = exact_gbm)
euro_call = options.EuropeanCall(strike = K)
mc_engine = engines.MonteCarloEngine(n_steps = n_steps, n_paths = n_paths, seed = seed)

price, std_error = mc_engine.price(
    euro_call,
    market,
    spot = s0,
    maturity = T
)

exact = bs.european_call(r = r, sigma = sigma, s = s0, K = K, T = T)

In [51]:
np.abs(price - exact), std_error

(np.float64(0.013142884672024735), np.float64(0.02085295543289006))